In [1]:
using HTTP
using CSV
using JSON3
import Dates
using DataFrames

# API INMET

A [API INMET](https://portal.inmet.gov.br/manual) possui diversos endpoints. Neste estudo, o objetivo é coletar e tratar séries temporais de estações meteorológicas.

In [2]:
function chamarAPI(url)
    try
        resposta = HTTP.get(url)
        return JSON3.read(String(resposta.body))
    catch e
        return "Um erro aconteceu: $(e)"
    end
end;

## Lista de estações

Para recuperar a lista de estações automáticas: https://apitempo.inmet.gov.br/estacoes/T

In [3]:
listaEstacoesAutomaticasJSON = chamarAPI("https://apitempo.inmet.gov.br/estacoes/T")
listaEstacoesAutomaticas = DataFrame(listaEstacoesAutomaticasJSON);

In [7]:
# nome das colunas
names(listaEstacoesAutomaticas);

In [8]:
# filtrando apenas estações operantes
listaEstacoesAutomaticasOperantes = listaEstacoesAutomaticas[
    listaEstacoesAutomaticas[:, "CD_SITUACAO"] .== "Operante", :];

## Dados por estação

Recuperar dados horários referentes a estação automática ou manual:
- https://apitempo.inmet.gov.br/estacao/{data_inicial}/{data_final}/{cod_estacao}
- https://apitempo.inmet.gov.br/estacao/2019-10-23/2019-10-23/A301
- Sendo data no formato AAAA-MM-DD (ex: 2019-10-23)

In [9]:
# os dados serão coletados posto a posto em um intervalo de datas
df = Dates.DateFormat("y-m-d")
dataInicial = Dates.Date("2021-01-01", df)
dataFinal = Dates.Date("2021-01-02", df)

# cria um diretório para salvar os dados obtidos
diretorioPersistencia = "dados_estacoes"
mkpath(diretorioPersistencia);

In [11]:
function coletarDadosEstacao(
        codigoEstacao::String,
        dataInicial::Dates.Date,
        dataFinal::Dates.Date)::DataFrame
    
    urlBaseDadosEstacoes = "https://apitempo.inmet.gov.br/estacao/"
    urlQueryDadosEstacoes = "$(dataInicial)/$(dataFinal)/$(codigoEstacao)"
    serieEstacaoJSON = chamarAPI("$(urlBaseDadosEstacoes)/$(urlQueryDadosEstacoes)")
    serieEstacao = DataFrame(serieEstacaoJSON)
    
    # substitui valores 'nothing' por 'missing'
    for coluna in names(serieEstacao)
        serieEstacao[!, coluna] = replace(
            serieEstacao[!, coluna], nothing => missing
        )
    end
    
    return serieEstacao
end;

In [13]:
codigosEstacoes = listaEstacoesAutomaticasOperantes[:, "CD_ESTACAO"]

for codigoEstacao in codigosEstacoes
    try
        serieEstacao = coletarDadosEstacao(codigoEstacao, dataInicial, dataFinal)
        CSV.write("$(diretorioPersistencia)/$(codigoEstacao).csv", serieEstacao)
        println("Dados da estação '$(codigoEstacao)' obtidos.")
    catch e
        println("Não foi possível processar os dados de '$(codigoEstacao)'. $e")
    end
end

Dados da estação 'A756' obtidos.
Dados da estação 'A045' obtidos.
Dados da estação 'A549' obtidos.
Dados da estação 'A826' obtidos.
Dados da estação 'A615' obtidos.
Dados da estação 'A053' obtidos.
Dados da estação 'A508' obtidos.
Dados da estação 'A909' obtidos.
Dados da estação 'A024' obtidos.
Dados da estação 'A934' obtidos.
Dados da estação 'A336' obtidos.
Dados da estação 'A434' obtidos.
Dados da estação 'S701' obtidos.
Dados da estação 'A340' obtidos.
Dados da estação 'A409' obtidos.
Dados da estação 'A566' obtidos.
Dados da estação 'A013' obtidos.
Dados da estação 'A054' obtidos.
Dados da estação 'S702' obtidos.
Dados da estação 'A867' obtidos.
Dados da estação 'A505' obtidos.
Dados da estação 'A309' obtidos.
Dados da estação 'A310' obtidos.
Dados da estação 'A736' obtidos.
Dados da estação 'A606' obtidos.
Dados da estação 'A725' obtidos.
Dados da estação 'A220' obtidos.
Dados da estação 'A375' obtidos.
Dados da estação 'S113' obtidos.
Dados da estação 'A565' obtidos.
Dados da e